In [0]:
storage_account = "adbdatalake01111"

spark.conf.set(
    f"fs.azure.account.auth.type.{storage_account}.dfs.core.windows.net",
    "OAuth"
)

spark.conf.set(
    f"fs.azure.account.oauth.provider.type.{storage_account}.dfs.core.windows.net",
    "org.apache.hadoop.fs.azurebfs.oauth2.MsiTokenProvider"
)

raw_path = f"abfss://raw@{storage_account}.dfs.core.windows.net/movies_parquet"
bronze_path = f"abfss://bronze@{storage_account}.dfs.core.windows.net/movies"


In [0]:
movies_df = spark.read.parquet(f"{raw_path}/movies")
film_details_df = spark.read.parquet(f"{raw_path}/film_details")
more_info_df = spark.read.parquet(f"{raw_path}/more_info")
poster_path_df = spark.read.parquet(f"{raw_path}/poster_path")


In [0]:
from pyspark.sql.functions import col

def clean_columns(df):
    for column in df.columns:
        df = df.withColumnRenamed(
            column,
            column.lower().replace(" ", "_")
        )
    return df


In [0]:
movies_clean = clean_columns(movies_df).dropDuplicates()
film_details_clean = clean_columns(film_details_df).dropDuplicates()
more_info_clean = clean_columns(more_info_df).dropDuplicates()
poster_path_clean = clean_columns(poster_path_df).dropDuplicates()


In [0]:
movies_clean.write.mode("overwrite").parquet(
    f"{bronze_path}/movies"
)

film_details_clean.write.mode("overwrite").parquet(
    f"{bronze_path}/film_details"
)

more_info_clean.write.mode("overwrite").parquet(
    f"{bronze_path}/more_info"
)

poster_path_clean.write.mode("overwrite").parquet(
    f"{bronze_path}/poster_path"
)
